In [1]:
from model_coupled import CoupledBert
from classifier import Classifier
from nli import NLIModel
from transformers import BertModel, AutoTokenizer
from utils import load_model
import numpy as np

cbert_tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
cbert_tokenizer.add_tokens(['[QUERY]','[DESCRIPTION]','[NUM]'])
cbert = BertModel.from_pretrained("sbert107")
cbert.resize_token_embeddings(cbert.embeddings.word_embeddings.weight.shape[0]+3)


cmodel = Classifier(cbert,cbert_tokenizer,mask_prob=0)
cmodel.eval()

load_model(cmodel,"gg_/checkpoint-1200")
cmodel.requires_grad_(False)


bert_tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
bert_tokenizer.add_tokens(['[QUERY]','[DESCRIPTION]','[NUM]'])
bert = BertModel.from_pretrained("10-8nli")
bert.eval()
top_k = 50

model = NLIModel(bert,bert_tokenizer,embedding_dim=512,k=top_k,mask_prob=0)
load_model(model,"10-8mean_sbert8___/checkpoint-404")
bert.requires_grad_(False)



model = CoupledBert(model,bert_tokenizer,top_k,50,cmodel,cbert_tokenizer).cuda()
model.eval()
model.prepare_description_tokens()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


trainable params: 131,716,096 || all params: 434,025,472 || trainable%: 30.3475


c:\dls-workshop\utils.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p = torch.load(f"{cp}/{name}").requires_grad_(False)


trainable params: 589,824 || all params: 427,501,568 || trainable%: 0.1380


c:\dls-workshop\utils.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p = torch.load(f"{cp}/{name}").requires_grad_(False)
Asking to truncate to max_length but no maximu

In [2]:
from data import train_dataset,test_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch


def collate(x):
    out = {"text":[],"labels":[]}
    for i in x:
        out['text'].append(i['text'])
        out['labels'].append(i['labels'])
    return out

test_loader = DataLoader(test_dataset,batch_size=64,collate_fn=collate)


test_preds = []
test_labels = []
test_sims = []
test_classes = []
test_descs = []
tp = []
test_embeddings = []
model.eval()
for xx in tqdm(test_loader):
    with torch.no_grad():
        pred = model.predict(xx['text'],k=top_k,labels=torch.Tensor(xx['labels']))
        cls_token,desc_token,classes,preds,labels = pred
        desc_token = desc_token.view(cls_token.shape[0],-1,512)
        test_classes.extend(classes.detach().cpu().numpy().tolist())
        for i in range(cls_token.shape[0]):
            test_sims.extend(torch.nn.functional.cosine_similarity(cls_token[i],desc_token[i]).detach().cpu().numpy().tolist())
        test_descs.extend(desc_token.detach().cpu().numpy().tolist())
        test_preds.extend(cls_token.detach().cpu().numpy().tolist())
        tp.extend(preds.detach().cpu().numpy().tolist())
        test_labels.extend(xx['labels'])

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 15/15 [00:07<00:00,  1.93it/s]


In [3]:
test_descs = np.array(test_descs).squeeze()
test_classes = np.array(test_classes).squeeze()
test_labels = np.array(test_labels).astype(int).squeeze()
test_preds = np.array(test_preds).squeeze()
test_y = []
test_classes = test_classes.reshape(test_preds.shape[0],top_k)
test_x = np.stack((np.repeat(np.expand_dims(test_preds,1),top_k,1),test_descs.reshape(test_preds.shape[0],top_k,-1)),axis=-1).reshape(-1,top_k,1024)
for i in range(test_classes.shape[0]):
    test_y.append(test_labels[i][test_classes.astype(int)[i]])
test_y = np.array(test_y)

In [4]:
tp = np.array(tp)
test_labels = np.array(test_labels)
test_classes = np.array(test_classes).reshape(test_labels.shape[0],-1)

In [5]:
tp = torch.sigmoid(torch.Tensor(tp)).detach().cpu().numpy()

In [15]:
from sklearn.metrics import f1_score,classification_report
max_acc = 0
max_acc_f = 0
max_f1 = 0
max_t = np.array([0,0,0,0])
max_k = 1
max_tc = 0
r = dict()
ds = np.array(train_dataset['labels'])

weights = np.array([i/len(ds) for i in ds.sum(axis = 0)])
weights -= weights.min()
weights /= weights.max()


for k in range(1,51):
    t_labels = []
    t_classes = []
    miss = 0
    for i in range(test_labels.shape[0]):
        t_classes.append(tp[i].argsort()[-k:])
        t_miss = test_labels[i][tp[i].argsort()[:-k]].sum()>0
        t_labels.append(test_labels[i][t_classes[-1]].tolist())
        if t_miss:
            miss+=1
            t_labels[-1]= [i*0 for i in t_labels[-1]]
    t_labels = np.array(t_labels)

    test_sims = np.array(test_sims).reshape(test_labels.shape)
    t_sims= []
    for i in range(test_labels.shape[0]):
        p = test_sims[i][np.argsort(test_classes[i])][tp[i].argsort()[-k:]]
        t_miss = test_labels[i][tp[i].argsort()[:-k]].sum()>0
        p[p.argmax()] = 1
        t_sims.append(p)
        if t_miss:
            t_sims[-1] = [1 for i in t_sims[-1]]
    t_sims = np.array(t_sims)
    
    
    from sklearn.metrics import accuracy_score    
    

    for thresh in tqdm(np.linspace(0,1,101)):
                
                acc = accuracy_score(t_labels.tolist(),(t_sims>thresh).astype(int).tolist())
                if  acc > max_acc:
                    max_acc = acc
                    max_acc_f = accuracy_score(t_labels.ravel().tolist(),((t_sims>thresh)).astype(int).ravel().tolist())
                    max_f1 = f1_score(t_labels.ravel().tolist(),((t_sims>thresh)).astype(int).ravel().tolist())
                    max_t = thresh.copy()
                    max_k = k
                    max_t = thresh

                    print("Accuracy: ",acc,"F1: ", max_f1,"Accuracy-flat: " ,max_acc_f, "top_k: ", max_k, f"threshold: ", thresh)
                    
                elif acc == max_acc:
                    f1 = f1_score(t_labels.ravel().tolist(),((t_sims*weights[t_classes[i]]>thresh)).astype(int).ravel().tolist())
                    if f1 > max_f1:
                        max_f1 = f1
                        max_acc_f = f1_score(t_labels.ravel().tolist(),((t_sims>thresh)).astype(int).ravel().tolist())
                        print("Accuracy: ",acc,"F1: ", max_f1,"Accuracy-flat: " ,max_acc_f, "top_k: ", max_k, f"threshold: ", thresh)
                        max_t = thresh.copy()
                        max_k = k
                        max_t = thresh


                    elif max_f1 == f1:
                        acc_f = accuracy_score(t_labels.ravel().tolist(),((t_sims>thresh)).astype(int).ravel().tolist())
                        if acc_f > max_acc_f:
                            max_acc_f = acc_f
                            print("Accuracy: ",acc,"F1: ", max_f1,"Accuracy-flat: " ,max_acc_f, "top_k: ", max_k, f"threshold: ", thresh)
                            max_t = thresh.copy()
                            max_k = k
                            max_t = thresh

100%|██████████| 101/101 [00:00<00:00, 509.18it/s]

Accuracy:  0.5145945945945946 F1:  0.6795146324054246 Accuracy-flat:  0.5145945945945946 top_k:  1 threshold:  0.0



100%|██████████| 101/101 [00:00<00:00, 508.77it/s]


Accuracy:  0.5286486486486487 F1:  0.6714997734481196 Accuracy-flat:  0.6081081081081081 top_k:  2 threshold:  0.76
Accuracy:  0.547027027027027 F1:  0.674301420064132 Accuracy-flat:  0.6156756756756757 top_k:  2 threshold:  0.77
Accuracy:  0.5675675675675675 F1:  0.6777932313398238 Accuracy-flat:  0.6243243243243243 top_k:  2 threshold:  0.78
Accuracy:  0.572972972972973 F1:  0.6747777257838091 Accuracy-flat:  0.6243243243243243 top_k:  2 threshold:  0.79
Accuracy:  0.5772972972972973 F1:  0.6701372456223379 Accuracy-flat:  0.6232432432432432 top_k:  2 threshold:  0.8
Accuracy:  0.5827027027027027 F1:  0.6660287081339713 Accuracy-flat:  0.6227027027027027 top_k:  2 threshold:  0.81
Accuracy:  0.5881081081081081 F1:  0.6653790439401256 Accuracy-flat:  0.6254054054054055 top_k:  2 threshold:  0.8200000000000001


100%|██████████| 101/101 [00:00<00:00, 891.15it/s]


...


 62%|██████▏   | 63/101 [00:00<00:00, 629.06it/s]

Accuracy:  0.6108108108108108 F1:  0.6444444444444445 Accuracy-flat:  0.8517374517374517 top_k:  7 threshold:  0.8200000000000001


100%|██████████| 101/101 [00:00<00:00, 415.84it/s]


Accuracy:  0.6118918918918919 F1:  0.6451378809869376 Accuracy-flat:  0.9186694386694386 top_k:  13 threshold:  0.8200000000000001


100%|██████████| 101/101 [00:00<00:00, 151.19it/s]
